## आवश्यक लायब्ररी आयात करा
Phi-4 मॉडेल लोड आणि वापरण्यासाठी आवश्यक PyTorch आणि transformers लायब्ररी आयात करा.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## रँडम बीज सेट करा
वेगवेगळ्या रनमध्ये पुनरुत्पादक परिणाम सुनिश्चित करण्यासाठी रँडम बीज सेट करा.


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning मॉडेल आणि टोकनायझर लोड करा
Microsoft च्या Phi-4-mini-flash-reasoning मॉडेल आणि त्यासोबतचा टोकनायझर Hugging Face वरून लोड करा. मॉडेल जलद अनुमानासाठी CUDA वर लोड केले जाईल.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## इनपुट संदेश तयार करा  
मॉडेलसाठी चॅट टेम्पलेट वापरून एक गप्पांचा संदेश तयार करा ज्यामध्ये द्विघात समीकरणाचा गणिताचा प्रश्न असेल.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## प्रतिसाद तयार करा
निर्दिष्ट पॅरामीटर्स वापरून मॉडेलकडून प्रतिसाद तयार करा, जसे की temperature आणि top_p, ज्यामुळे आउटपुटमधील randomness नियंत्रित करता येते.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## आउटपुट टेक्स्टमध्ये डिकोड करा
निर्मित टोकन अनुक्रमांना परत मानवी-वाचनीय मजकूरामध्ये रूपांतरित करा, मूळ इनपुट टोकन वगळून फक्त मॉडेलचा प्रतिसाद दर्शवा.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
